# 10 apply 메서드 활용

## 10-1 간단한 함수 만들기

### 제곱함수와 n제곱함수 만들기

In [1]:
# 제곱함수
def my_sq(x) :
    return x**2

In [2]:
# n제곱함수
def my_exp(x, n) :
    return x**n

In [3]:
print(my_sq(4))

16


In [4]:
print(my_exp(2,4))

16


## 10-2 apply 메서드 사용하기 - 기초

### 시리즈와 데이터프레임에 apply 메서드 사용하기

In [5]:
# 데이터프레임 준비
import pandas as pd

df = pd.DataFrame({'a' : [10, 20, 30], 'b' : [20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [6]:
# a열을 제곱하여 얻은 결과
print(df['a']**2)

0    100
1    400
2    900
Name: a, dtype: int64


In [7]:
# 위에 만든 my_sq 제곱함수 적용하기
    # 1개의 인자를 받도록 구성되어 있으면 인잣값 생략
sq = df['a'].apply(my_sq)
print(sq)

0    100
1    400
2    900
Name: a, dtype: int64


In [8]:
# 2개의 인자를 전달받아야 할 때
    # apply 메서드 -> 첫번째 인자 : n 제곱함수의 이름, 두번째 인자 : n 제곱함수의 두번째 인자(n) 전달
ex = df['a'].apply(my_exp, n=2)
print(ex)

0    100
1    400
2    900
Name: a, dtype: int64


In [9]:
ex = df['a'].apply(my_exp, n=3)
print(ex)

0     1000
1     8000
2    27000
Name: a, dtype: int64


< 데이터프레임과 apply 메서드 >

In [10]:
# 데이터프레임 준비
df = pd.DataFrame({'a' : [10, 20, 30], 'b' : [20, 30, 40]})
print(df)

    a   b
0  10  20
1  20  30
2  30  40


In [11]:
#  함수만들기 -> 1개의 값을 전달받아 출력
def print_me(x) :
    print(x)

In [18]:
# axis 인잣값을 이용하여 함수를 열방향 또는 행방향으로 적용할지 정하기
    # 인잣값 0 : 열(기본), 1 : 행
print(df.apply(print_me, axis=0))

0    10
1    20
2    30
Name: a, dtype: int64
0    20
1    30
2    40
Name: b, dtype: int64
a    None
b    None
dtype: object


In [14]:
print(df['a'])

0    10
1    20
2    30
Name: a, dtype: int64


In [15]:
print(df['b'])

0    20
1    30
2    40
Name: b, dtype: int64


In [16]:
print(df.apply(print_me, axis=1))

a    10
b    20
Name: 0, dtype: int64
a    20
b    30
Name: 1, dtype: int64
a    30
b    40
Name: 2, dtype: int64
0    None
1    None
2    None
dtype: object


In [17]:
# 3개의 인자를 입력받아 평균 계산하는 함수 만들기
def avg_3(x, y, z) :
    return(x + y + z) / 3

In [19]:
print(df.apply(avg_3))

TypeError: ("avg_3() missing 2 required positional arguments: 'y' and 'z'", 'occurred at index a')

In [21]:
# 3개의 인잣값 필요한데 1개만 입력받았다는 오류메시지
# 열단위로 데이터가 전달되었다(df['a']또는  df['b'])
# -> 열단위로 데이터 처리할 수 있도록 수정해야함

In [24]:
# 위 함수가 열단위로 데이터 처리할 수 있도록 개선한 함수
def avg_3_apply(col):
    x = col[0]
    y = col[1]
    z = col[2]
    return (x + y + z) / 3

print(df.apply(avg_3_apply))

a    20.0
b    30.0
dtype: float64


In [25]:
# for문으로 작성하기
def avg_3_apply(col) :
    sum = 0
    for item in col :
        sum += item
    return sum / df.shape[0]

In [26]:
# 위를 응용하여 행방향으로 데이터 처리하기
    # df.shape[1]로만 바꾸면 된다.
def avg_2_apply(row) :
    sum = 0
    for item in row :
        sum += item
    return sum / df.shape[1]

print(df.apply(avg_2_apply, axis = 1))

0    15.0
1    25.0
2    35.0
dtype: float64


## apply 메서드 사용하기 - 고급

### 데이터프레임의 누락값을 처리한 다음 apply 메서드 사용하기

데이터프레임 누락값 처리하기 - 열방향

In [27]:
import seaborn as sns

titanic = sns.load_dataset("titanic")

In [28]:
print(titanic.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
survived       891 non-null int64
pclass         891 non-null int64
sex            891 non-null object
age            714 non-null float64
sibsp          891 non-null int64
parch          891 non-null int64
fare           891 non-null float64
embarked       889 non-null object
class          891 non-null category
who            891 non-null object
adult_male     891 non-null bool
deck           203 non-null category
embark_town    889 non-null object
alive          891 non-null object
alone          891 non-null bool
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB
None


In [29]:
# 누락값의 개수를 반환하는 함수
import numpy as np

def count_missing(vec) :
    null_vec = pd.isnull(vec) # isnull메서드 : 누락값의 유무에 따라 True, False를 적용한 데이터프레임 만들어진다.
    null_count = np.sum(null_vec) # sum메서드 : 누락값 개수 구하기
    return null_count

In [30]:
cmis_col = titanic.apply(count_missing)
print(cmis_col)

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64


In [31]:
# 누락값의 비율 계산하는 함수
def prop_missing(vec) :
    num = count_missing(vec) # 데이터프레임의 누락값 개수 구하기
    dem = vec.size # size 속성을 이용해 전체 데이터 수 구하기
    return num / dem

In [32]:
pmis_col = titanic.apply(prop_missing)
print(pmis_col)

survived       0.000000
pclass         0.000000
sex            0.000000
age            0.198653
sibsp          0.000000
parch          0.000000
fare           0.000000
embarked       0.002245
class          0.000000
who            0.000000
adult_male     0.000000
deck           0.772166
embark_town    0.002245
alive          0.000000
alone          0.000000
dtype: float64


In [33]:
# 누락값이 아닌 데이터의 비율 구하기
def prop_complete(vec) :
    return 1 - prop_missing(vec)

데이터프레임의 누락값 처리하기 - 행방향

In [34]:
# axis를 1로 설정하여 행방향으로 적용하기

cmis_row = titanic.apply(count_missing, axis=1)
pmis_row = titanic.apply(prop_missing, axis = 1)
pcom_row = titanic.apply(prop_complete, axis = 1)

print(cmis_row.head())

0    1
1    0
2    1
3    0
4    1
dtype: int64


In [35]:
print(pmis_row.head())

0    0.066667
1    0.000000
2    0.066667
3    0.000000
4    0.066667
dtype: float64


In [36]:
print(pcom_row.head())

0    0.933333
1    1.000000
2    0.933333
3    1.000000
4    0.933333
dtype: float64


In [37]:
# 누락값의 개수를 구하여 프레임에 추가하기
titanic['num_missing'] = titanic.apply(count_missing, axis = 1)

print(titanic.head())

   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  num_missing  
0    man        True  NaN  Southampton    no  False            1  
1  woman       False    C    Cherbourg   yes  False            0  
2  woman       False  NaN  Southampton   yes   True            1  
3  woman       False    C  Southampton   yes  False            0  
4    man        True  NaN  Southampton    no   True            1  


In [38]:
# 누락값이 2개 이상인 데이터 추출하기
print(titanic.loc[titanic.num_missing > 1, :].sample(10))

     survived  pclass     sex  age  sibsp  parch     fare embarked  class  \
464         0       3    male  NaN      0      0   8.0500        S  Third   
692         1       3    male  NaN      0      0  56.4958        S  Third   
274         1       3  female  NaN      0      0   7.7500        Q  Third   
198         1       3  female  NaN      0      0   7.7500        Q  Third   
888         0       3  female  NaN      1      2  23.4500        S  Third   
650         0       3    male  NaN      0      0   7.8958        S  Third   
868         0       3    male  NaN      0      0   9.5000        S  Third   
324         0       3    male  NaN      8      2  69.5500        S  Third   
420         0       3    male  NaN      0      0   7.8958        C  Third   
611         0       3    male  NaN      0      0   7.0500        S  Third   

       who  adult_male deck  embark_town alive  alone  num_missing  
464    man        True  NaN  Southampton    no   True            2  
692    man    